In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

In [2]:
# os.chdir('..')
base_dir = os.getcwd()

In [3]:
# img_dir = os.path.join(base_dir, 'dataset/images')
img_dir = os.path.join(base_dir, 'images')

In [4]:
base_dir

'/home/claudic/JetsonNano/eye_close_project/dataset'

In [5]:
# Define the data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the same size as expected by ResNet
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

dataset = datasets.ImageFolder(img_dir, transform=transform)

# Split the dataset into train, test, and validation sets
train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, val_size])

# Create data loaders for train, test, and validation sets
batch_size = 16  # Replace with your desired batch size

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

print(f"Train dataset size: {len(train_loader.dataset)}")
print(f"Test dataset size: {len(test_loader.dataset)}")
print(f"Validation dataset size: {len(val_loader.dataset)}")

Train dataset size: 56113
Test dataset size: 7014
Validation dataset size: 7015


In [6]:
dataset.class_to_idx

{'close': 0, 'open': 1}

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained ResNet-34 model
model = models.resnet34(pretrained=True)

# Modify the last fully connected layer for the desired number of classes
num_classes = 2  # Replace with the actual number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/claudic/anaconda3/envs/ecg-classification/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/claudic/anaconda3/envs/ecg-classification/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
print(device)

cuda


In [ ]:
# Train the model
num_epochs = 10  # Replace with the desired number of training epochs
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:  # Replace train_loader with your data loader
        inputs = inputs.to(device)  # Move inputs to the device
        labels = labels.to(device)  # Move labels to the device

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'resnet34_trained.pth')


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model.eval()  # Set the model to evaluation mode

y_true = []
y_pred = []

with torch.no_grad():  # Disable gradient calculation for validation
    for inputs, labels in val_loader:
        inputs = inputs.to(device)  # Move inputs to the device
        labels = labels.to(device)  # Move labels to the device

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the predicted labels

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
confusion_mat = confusion_matrix(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred)

print(f"Validation Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(classification_rep)

In [ ]:
import matplotlib.pyplot as plt

# Select a few random samples from the validation dataset
num_samples = 5
random_indices = np.random.choice(len(val_dataset), num_samples, replace=False)

# Plot the images along with their true and predicted labels
fig, axes = plt.subplots(nrows=1, ncols=num_samples, figsize=(15, 5))

for i, index in enumerate(random_indices):
    image, label = val_dataset[index]
    image = image.permute(1, 2, 0)  # Reshape image tensor for plotting
    predicted = model(image.unsqueeze(0).to(device)).argmax().item()

    axes[i].imshow(image)
    axes[i].set_title(f"True: {label}, Predicted: {predicted}")
    axes[i].axis('off')

plt.show()